# IQ Constellation Data Explore
This Notebook explores the following approach
1. Apply KMeans Clustering to map blobs to clusters
2. Extract Blob Shape by applying Covariance Error Ellipse
3. Plot new metrics next to raw blob plot (Optional)
4. Use statistical methods to determine Interference/Phase Noise/Compression/IQ Imablance/Normal features
5. Write features to tabular data format

Kernel Used: python3, ml.m5.xlarge

## Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import os

from utility import constellation_metrics

## Setting MODCOD and data classes

In [2]:
# Define all modulation types and data types
modcods = ['qpsk', '8psk', '16apsk', '32apsk']
data_types = ['normal', 'phase_noise', 'interference', 'compression', 'iq_imbalance']

## Processing IQ Constellations
1. Note - to visualize processing, set processor.process_file(file_path, data_type, **plot=True**)
    2. Because of the amount of plots generated, consider only plotting a single data_type/modcod
        3. modcods = ['qpsk']
        4. data_types = ['normal']
        5. plot=True)
2. Much of the processing comes from **utility/constellation_metrics.py**

In [ ]:
# List to store all results
all_results = []

# Process each combination
for modcod in modcods:
    print(f"\nProcessing {modcod.upper()} modulation...")
    processor = constellation_metrics.ConstellationProcessor(modcod=modcod)
    
    for data_type in data_types:
        print(f"  Processing {data_type} data...")
        base_path = f'../data_generation/generator/data/{modcod}/{data_type}'
        
        if not os.path.exists(base_path):
            print(f"    Warning: Path {base_path} does not exist. Skipping...")
            continue
        
        # Process each file in directory
        for filename in os.listdir(base_path):
            if filename.endswith('.csv'):
                file_path = os.path.join(base_path, filename)
                result = processor.process_file(file_path, data_type, plot=False)
                if result:
                    all_results.append(result)

In [4]:
# Convert to DataFrame
results_df = pd.DataFrame(all_results)

## Output metrics in tabular format

In [ ]:
# Save results
output_path = 'constellation_analysis_results.csv'
results_df.to_csv(output_path, index=False)
print(f"\nResults saved to {output_path}")

In [ ]:
# Display summary
print("\nResults Summary:")
print(results_df.head())